# Equipo 1: Pandalytics
* **A00832444** | Andrea Garza  
* **A01197991** | Hiram Maximiliano Muñoz Ramírez  
* **A00517124** | Erick Orlando Hernández Vallejo  
* **A01197655** | Raúl Isaí Murillo Alemán   
* **A01235692** | David Gerardo Martínez Hidrogo   

## Posibles variables a revisar
* Tipo de Pintura: Diferentes pinturas pueden tener distintos rendimientos debido a su composición.
* Condiciones de Aplicación: Temperatura, humedad, y presión durante la aplicación pueden afectar el rendimiento.
* Método de Aplicación: Rodillo, brocha, o spray pueden resultar en diferentes eficiencias.
* Superficie de Aplicación: La textura, material, y preparación de la superficie pueden influir en cuánta pintura se necesita.
* Espesor de Capa: El espesor deseado o aplicado de la pintura puede variar el rendimiento.
* Experiencia del Aplicador: La habilidad o técnica del pintor puede afectar la cantidad de pintura utilizada.
* Rendimiento Estándar y Real Previos: Datos históricos pueden ayudar a identificar tendencias o patrones en el rendimiento de la pintura.

In [1]:
import pandas as pd
import pyarrow.feather as feather
import uuid

Primero se cargarón dos de los archivos de excel: los datos sobre las pinturas y revestidos de julio 20 a agosto 23, y el análisis de consumo de pintura.

In [2]:
#coating_df = pd.read_excel('data/pinturas_revestidos_jul20_ago23.xlsx', sheet_name='Pintado 1 UNI Junio', header=1)
coating_df = pd.read_excel('data/pinturas_revestidos_jul20_ago23.xlsx', sheet_name='Pintado 1 UNI Ago')

paint_analysis_df = pd.read_excel('data/analisis_consumo_pintura.xlsx', sheet_name='Base de datos por pintura')

Los datos se guardaron en archivos formato Feather, para permitir una
carga de datos mas rápida.
Se necesitó hacer una asignación manual de tipos en dos columnas, ya que el algoritmo de Feather no logró intuir los tipos de estas columnas.

In [3]:
coating_df['Nº doc.'] = coating_df['Nº doc.'].astype(str)
coating_df['Asignación'] = coating_df['Asignación'].astype(str)
feather.write_feather(coating_df, 'data/pinturas_revestidos_jul20_ago23.feather')
feather.write_feather(paint_analysis_df, 'data/analisis_consumo_pintura.feather')

In [4]:
coating_df = pd.read_feather('data/pinturas_revestidos_jul20_ago23.feather')
paint_analysis_df = pd.read_feather('data/analisis_consumo_pintura.feather')

## Análisis de datos de pinturas y revestidos
Primero se empezó por obter la estructura general del conjunto de datos.

El conjunto de datos tiene un total de 211,714 filas. Tiene 43 columnas diferentes. En total, el dataframe ocupa alrededor de 70 MB de memoria.

A continuación se muestra la información anteriormente mencionada, así como los tipos de cada columna y los valores no nulos de cada columna.

Para este conjunto de datos consideramos que N°doc.ref. sería mejor manejado como string y no como int ya que es un identificador, además Fecha., Fe.contab. y Registro serían mejor manejados como tipo datetime.

Existen también múltiples columnas que están siendo manejadas por defecto como tipo float ya que no cuenten con registros.

Se encontro que el nombre de columna "Año" esta repetido 

In [5]:
coating_df.info()

In [6]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(coating_df.isna().sum() / coating_df.shape[0])

In [7]:
coating_empty_cols = [
    'OrdPartner',
    'Doc.compr.',
    'Pos.',
    'Txt.pedido',
    'Acreedor',
    'Nom.1',
]

Se calculó un resumen estadístico descriptivo de las columnas numéricas, excluyendo las que no cuentan con registros:

In [8]:
coating_analysis_cols = [
    'Ce.coste',
    'Per',
    ' Año', 
    '  Valor/Mon.tr.',
    ' Valor/Mon.obj.', 
    '    Valor var.', 
    'Ctd.total reg.',
    'Precio'
]

coating_df[coating_analysis_cols].describe()

## Análisis de datos del consumo de pintura

Primero se empezó por obtener la estructura general del conjunto de datos.

El conjunto de datos tiene un total de 566 filas. Tiene 15 columnas diferentes. En total, el dataframe ocupa alrededor de 67 KB de memoria.

A continuación se muestra la información anteriormente mencionada, así como los tipos de cada columna y los valores no nulos de cada columna.

Para este conjunto de datos consideramos que Linea, Mes 2, Mes num y Mes serían mejor manejados como datos categoricos.

Para el resto de columnas consideramos que su tipo de dato es adecuado, incluyendo las que no cuentan con registros.

In [9]:
paint_analysis_df.info()

In [10]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(paint_analysis_df.isna().sum() / paint_analysis_df.shape[0])

In [11]:
paint_analysis_empty_cols = ['Error de Consumo', 'Diferencia porcentual de rendimiento']

Se calculó un resumen estadístico descriptivo de las columnas numéricas, excluyendo las que no cuentan con registros:

In [12]:
paint_analysis_cols = [
    'Real', 
    'Teo', 
    'Dif', 
    'Magnitud', 
    'Rendimeinto Std', 
    'Metros cuadrados reales', 
    'Rendimeinto Real', 
    'Diferencia de Rendimiento', 
]

paint_analysis_df[paint_analysis_cols].describe()

Finalmente, se elaboro un conteo de valores unicos en cada una de las demas columnas.

Linea, Mes 2, Mes y num son probablemente categorías.
Los demas son valores de string únicos.

In [13]:
paint_analysis_df[filter(lambda x: x not in paint_analysis_cols and x not in paint_analysis_empty_cols, paint_analysis_df.columns)].nunique()

Categorías de las columnas mencionadas anteriormente:

In [14]:
paint_analysis_df[['Linea ', 'Mes 2', 'Mes num','Mes']].apply(pd.unique)

## Resumenes de líneas de producción y defectos
Tambíen cargamos todos los dataframes de los registros de líneas de producción:

In [15]:
april_production_1_df = pd.read_excel('data/paint_production_summaries/april_1.xlsx', sheet_name='ResumenProduccion')
april_production_2_df = pd.read_excel('data/paint_production_summaries/april_2.xlsx', sheet_name='ResumenProduccion')
may_production_1_df = pd.read_excel('data/paint_production_summaries/may_1.xlsx', sheet_name='ResumenProduccion')
may_production_2_df = pd.read_excel('data/paint_production_summaries/may_2.xlsx', sheet_name='ResumenProduccion')
june_production_1_df = pd.read_excel('data/paint_production_summaries/june_1.xlsx', sheet_name='ResumenProduccion')
june_production_2_df = pd.read_excel('data/paint_production_summaries/june_2.xlsx', sheet_name='ResumenProduccion')
july_production_1_df = pd.read_excel('data/paint_production_summaries/july_1.xlsx', sheet_name='ResumenProduccion')
july_production_2_df = pd.read_excel('data/paint_production_summaries/july_2.xlsx', sheet_name='ResumenProduccion')
august_production_1_df = pd.read_excel('data/paint_production_summaries/august_1.xlsx', sheet_name='ResumenProduccion')
august_production_2_df = pd.read_excel('data/paint_production_summaries/august_2.xlsx', sheet_name='ResumenProduccion')


#april_production_1_df[''] = april_production_1_df[''].astype(str)
# feather.write_feather(april_production_1_df, 'data/april_1.feather')
# feather.write_feather(april_production_2_df, 'data/april_2.feather')
# feather.write_feather(may_production_1_df, 'data/may_1.feather')
# feather.write_feather(may_production_2_df, 'data/may_2.feather')
# feather.write_feather(june_production_1_df, 'data/june_1.feather')
# feather.write_feather(june_production_2_df, 'data/june_2.feather')
# feather.write_feather(july_production_1_df, 'data/july_1.feather')
# feather.write_feather(july_production_2_df, 'data/july_2.feather')
# feather.write_feather(august_production_1_df, 'data/august_1.feather')
# feather.write_feather(august_production_2_df, 'data/august_2.feather')

Se definieron funciones para procesar y separar los datos de cada línea de producción de los defectos en cada una.

In [16]:
defects_cols = [
        'Codigo defecto',
        'Defecto',
        'Ubicacion',
        'Es Contencion',
        'Es Prevencion',
        'Es Mejora',
        'Intensidad',
        'Cara',
        'Lado',
        'Linea Origen',
        'Dictamen',
        'Resolución',
        'Calidad Sugerida',
        'Siguiente Accion',
        'Siguiente Proceso',
        'Mts Ini',
        'Mts Fin',
        'Frecuencia',
        'Causa',
        'Accion Preventiva',
        'Accion Correctiva',
        'Observaciones',
        'Fecha Registro',
        'Registro Automatico',
        'Código Equipo',
]

def reindex_production_df(production_df: pd.DataFrame):
    df = production_df.assign(production_id=production_df.apply(lambda x: uuid.uuid4(), axis=1))
    df.set_index('production_id', inplace=True)
    return df

def extract_defects_df(production_df: pd.DataFrame):
    defects_df_1 = production_df[map(lambda x: x + ' (1)', defects_cols)].dropna(thresh=1)
    defects_df_1.columns = defects_cols
    defects_df_2 = production_df[map(lambda x: x + ' (2)', defects_cols)].dropna(thresh=1)
    defects_df_2.columns = defects_cols
    defects_df_3 = production_df[map(lambda x: x + ' (3)', defects_cols)].dropna(thresh=1)
    defects_df_3.columns = defects_cols
    defects_df_4 = production_df[map(lambda x: x + ' (4)', defects_cols)].dropna(thresh=1)
    defects_df_4.columns = defects_cols
    defects_df_5 = production_df[map(lambda x: x + ' (5)', defects_cols)].dropna(thresh=1)
    defects_df_5.columns = defects_cols
    
    defects_df = pd.concat([
        defects_df_1.reset_index(names='production_id'), 
        defects_df_2.reset_index(names='production_id'), 
        defects_df_3.reset_index(names='production_id'),
        defects_df_4.reset_index(names='production_id'),
        defects_df_5.reset_index(names='production_id'),
    ], ignore_index=True)

    defects_df['defect_id'] = defects_df.apply(lambda x: uuid.uuid4(), axis=1)
    defects_df.set_index('defect_id', inplace=True)

    return defects_df

def remove_defects_cols(production_id: pd.DataFrame):
    columns_to_remove = list(map(lambda x: x + ' (1)', defects_cols)) + list(map(lambda x: x + ' (2)', defects_cols)) + list(map(lambda x: x + ' (3)', defects_cols)) + list(map(lambda x: x + ' (4)', defects_cols)) + list(map(lambda x: x + ' (5)', defects_cols))
    return production_id.drop(columns=columns_to_remove)
    

april_production_df = pd.concat([
    reindex_production_df(april_production_1_df),
    reindex_production_df(april_production_2_df),
])
april_defects_df = extract_defects_df(april_production_df)
april_production_df = remove_defects_cols(april_production_df)

may_production_df = pd.concat([
    reindex_production_df(may_production_1_df),
    reindex_production_df(may_production_2_df),
])
may_defects_df = extract_defects_df(may_production_df)
may_production_df = remove_defects_cols(may_production_df)

june_production_df = pd.concat([
    reindex_production_df(june_production_1_df),
    reindex_production_df(june_production_2_df),
])
june_defects_df = extract_defects_df(june_production_df)
june_production_df = remove_defects_cols(june_production_df)

july_production_df = pd.concat([
    reindex_production_df(july_production_1_df),
    reindex_production_df(july_production_2_df),
])
july_defects_df = extract_defects_df(july_production_df)
july_production_df = remove_defects_cols(july_production_df)

august_production_df = pd.concat([
    reindex_production_df(august_production_1_df),
    reindex_production_df(august_production_2_df),
])
august_defects_df = extract_defects_df(august_production_df)
august_production_df = remove_defects_cols(august_production_df)

production_df = pd.concat([
    april_production_df,
    may_production_df,
    june_production_df,
    july_production_df,
    august_production_df,
])
defects_df = pd.concat([
    april_defects_df,
    may_defects_df,
    june_defects_df,
    july_defects_df,
    august_defects_df,
])

In [17]:
production_df.index = production_df.index.astype(str)
defects_df.index = defects_df.index.astype(str)
defects_df['production_id'] = defects_df['production_id'].astype(str)
feather.write_feather(production_df, 'data/production.feather')
feather.write_feather(defects_df, 'data/defects.feather')

In [18]:
production_df = pd.read_feather('data/production.feather')
defects_df = pd.read_feather('data/defects.feather')

### Análisis de datos de producción

Primero obtenemos obtenemos la estructura general de la base de datos

El conjunto de datos tiene un total de 17650 filas. Tiene 130 columnas diferentes. En total, el dataframe ocupa alrededor de 18 MB de memoria.

A continuación se muestra la información anteriormente mencionada, así como los tipos de cada columna y los valores no nulos de cada columna.

In [19]:
production_df.info()

In [20]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
   display(production_df.dtypes)

In [21]:
production_empty_cols = [
    'Balance Forzado',
    'Usuario Balance Peso',
    'Usuario Registro Peso'
]

Se calculó un resumen estadístico descriptivo de las columnas numéricas, excluyendo las que no cuentan con registros:

In [22]:
production_analysis_cols = [
    'Fecha Inicio', 'Fecha Fin', 'Turno', 'Necesidad', 'Ancho 1', 'Ancho 2',
    'Ancho 3', 'Ancho', 'Espesor 1', 'Espesor 2', 'Espesor 3', 'Espesor',
    'Peso Entrada', 'Peso', 'Largo', 'Largo PLC', 'Peso Mínimo',
    'Peso Máximo', 'Calibre', 'Código Primer Inf', 'Código Primer Sup',
    'Capa Acabado Inf', 'Capa Acabado LO Inf', 'Capa Acabado LM Inf',
    'Capa Acabado Sup', 'Capa Acabado LO Sup', 'Capa Acabado LM Sup',
    'Capa Primer Inf', 'Capa Primer LO Inf', 'Capa Primer LM Inf',
     'Capa Primer Sup', 'Capa Primer LO Sup', 'Capa Primer LM Sup',
     'Capa Clear Inf', 'Capa Clear LO Inf', 'Capa Clear MN Inf',
     'Capa Clear Sup', 'CapaClear LO Sup', 'Capa Clear LM Sup', 'Brillo Sup',
     'Brillo Inf', 'Grado Brillo Sup', 'Grado Brillo Inf',
     'Impacto Pintura Inf', 'Impacto Pintura LO Inf',
     'Impacto Pintura LM Inf', 'Impacto Pintura Sup',
     'Impacto Pintura LO Sup', 'Impacto Pintura LM Sup',
     'Resistencia Solvente Sup', 'Resistencia Solvente Inf',
     'Análisis Tono DA Inf', 'Análisis Tono DA Sup', 'Análisis Tono DB Inf',
     'Análisis Tono DB Sup', 'Análisis Tono DC Inf', 'Análisis Tono DC Sup',
     'Análisis Tono DE Inf', 'Análisis Tono DE Sup', 'Análisis Tono DL Inf',
     'Análisis Tono DL Sup', 'Polietileno Metros Aplicados',
     'Polietileno Ancho', 'Velocidad Promedio', '% Línea Operando',
     '% Primer Sup. Salida', '% Primer Inf. Salida', '% Cromato Inf Salida',
     '% Cromato Sup Salida', '% Acabado 1 Inf Salida',
     '% Acabado 1 Sup Salida', '% Uso Cabezales', 'Programa',
     'Vel. Proceso Promedio', 'Es Reproceso', '% Acabado Inf Salida',
     '% Acabado Sup Norte Salida', '% Acabado Sup Sur Salida'
     ]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(production_df[production_analysis_cols].describe())

Finalmente, se elaboro un conteo de valores unicos en cada una de las demas columnas.

Estado, Clase, Norma, Uso General, Empaque, las durezas de pintura, la felxión de la pintura, Cinta de Noche, Hizo Proceso NO STD, Dictamen, Muestrear, Muestreado, Hizo Proceso NO STD son probablemente categorías. Los demas son valores de string únicos.

In [23]:
production_df[filter(lambda x: x not in production_analysis_cols and x not in production_empty_cols, production_df.columns)].nunique()

### Análisis de defectos

Primero obtenemos obtenemos la estructura general de la base de datos

El conjunto de datos tiene un total de 17650 filas. Tiene 130 columnas diferentes. En total, el dataframe ocupa alrededor de 18 MB de memoria.

A continuación se muestra la información anteriormente mencionada, así como los tipos de cada columna y los valores no nulos de cada columna.

In [24]:
defects_df.info()

In [25]:
defects_empty_cols = [
    'Siguiente Accion',
    'Siguiente Proceso',
    'Mts Ini',
    'Mts Fin'
]

Se calculó un resumen estadístico descriptivo de las columnas numéricas, excluyendo las que no cuentan con registros:

In [26]:
defects_analysis_cols = [
    'Codigo defecto', 
    'Es Contencion', 
    'Es Prevencion', 'Es Mejora',
    'Intensidad', 
    'Fecha Registro', 
    'Registro Automatico'
]
defects_df[defects_analysis_cols].describe()

Finalmente, se elaboro un conteo de valores unicos en cada una de las demas columnas.

Ubicación, Cara, Lado, Dictamen, Resolución, Calidad Sugerida, Frecuencia son probablemente categorías. Los demas son valores de string únicos.

In [27]:
defects_df[filter(lambda x: x not in defects_analysis_cols and x not in defects_empty_cols, defects_df.columns)].nunique()

## Columnas comunes que permitan enlazar registros

La columna Color del dataframe de análisis de pintura podría relacionarse con las columnas de Color Superior y Color Inferior del dataframe de Producción